In [23]:
import dask.dataframe as dd
import pandas as pd

path_files = "data/layouts/"

# Define os nomes das colunas das tabelas
col_simples = ['CNPJ_Basico', 'Opcao_Simples', 'Data_Opcao_Simples', 'Data_Exclusao_Simples', 'Opcao_MEI', 'Data_Opcao_MEI', 'Data_Exclusao_MEI']
col_default = ['Codigo', 'Descricao']

In [24]:
df_simples = dd.read_csv(f'{path_files}*.SIMPLES.CSV*.zip', delimiter=';', encoding='latin1', low_memory=False, dtype=str, names=col_simples, blocksize=None)
df_cnae = dd.read_csv(f'{path_files}*.CNAECSV.zip', delimiter=';', encoding='latin1', low_memory=False, dtype=str, names=col_default, blocksize=None)
df_municipio = dd.read_csv(f'{path_files}*.MUNICCSV.zip', delimiter=';', encoding='latin1', low_memory=False, dtype=str, names=col_default, blocksize=None)
df_natureza = dd.read_csv(f'{path_files}*.NATJUCSV.zip', delimiter=';', encoding='latin1', low_memory=False, dtype=str, names=col_default, blocksize=None)
df_pais = dd.read_csv(f'{path_files}*.PAISCSV.zip', delimiter=';', encoding='latin1', low_memory=False, dtype=str, names=col_default, blocksize=None)


In [25]:
# reparticiona para reduzir o tamanho dos arquivos parquet
df_simples = df_simples.repartition(npartitions=3)

In [ ]:
df_simples['Data_Opcao_Simples'] = pd.to_datetime(df_simples['Data_Opcao_Simples'], errors='coerce')

In [17]:
df_simples['Data_Opcao_Simples'] = df_simples['Data_Opcao_Simples'].replace(["00000000"], None)
df_simples['Data_Exclusao_Simples'] = df_simples['Data_Exclusao_Simples'].replace(["00000000"], None)
df_simples['Data_Opcao_MEI'] = df_simples['Data_Opcao_MEI'].replace(["00000000"], None)
df_simples['Data_Exclusao_MEI'] = df_simples['Data_Exclusao_MEI'].replace(["00000000"], None)

In [18]:
# df_simples['Data_Opcao_Simples'] = df_simples['Data_Opcao_Simples'].astype('datetime64')
# df_simples['Data_Exclusao_Simples'] = df_simples['Data_Exclusao_Simples'].astype('datetime64')
# df_simples['Data_Opcao_MEI'] = df_simples['Data_Opcao_MEI'].astype('datetime64')
# df_simples['Data_Exclusao_MEI'] = df_simples['Data_Exclusao_MEI'].astype('datetime64')

In [19]:
import gcsfs
my_gs_bucket = 'dna_public_cnpj_files/parquet_files/'
token_json = 'auth/dados-publicos-326316-e16540376290.json'

In [20]:
def upload_dataframe_to_gcs(df, bucket, token, directory):
  try:
    df.to_parquet(f'gs://{bucket}{directory}', write_index=False, storage_options={'token': token})
  except Exception as e:
    raise e

In [21]:
df_dict = {'simples': df_simples, 'cnae': df_cnae, 'municipio': df_municipio, 'nat_juridica': df_natureza, 'pais': df_pais}

In [ ]:
for df in df_dict:
  upload_dataframe_to_gcs(df_dict[df], my_gs_bucket, token_json, df)